In [ ]:
import zipfile
import os
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
import keras
import numpy as np
import copy

In [ ]:
# распаковка архивов
zip = zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip', 'r')
zip.extractall("/kaggle/working/train_extracted")

In [ ]:
zip = zipfile.ZipFile('/kaggle/input/dogs-vs-cats/test1.zip', 'r')
zip.extractall("/kaggle/working/test_extracted")

In [ ]:
# функция для представления картинки в нужном формате
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x   

In [ ]:
# получаем имена всех файлов
walk = os.walk("/kaggle/working/train_extracted")
for f in walk:
   filenames = f[2]
filenames[:10]

In [ ]:
len(filenames)

In [ ]:
# важные настройки для обучения
category_encoding = {"cat": 0, "dog": 1}

In [ ]:
# разделяем имена файлов с котиками и пёсиками
categories = {"cat": [], "dog": []}
for filename in filenames:
    if "cat" in filename:
        categories["cat"].append(filename)
    else:
        categories["dog"].append(filename)
        
print(len(categories["cat"]), len(categories["dog"]))

In [ ]:
# перемешиваем
np.random.shuffle(categories["cat"])
np.random.shuffle(categories["dog"])

In [ ]:
# соединяем по 500 картинок для обучающей выборки
work_filenames = copy.copy(categories["cat"][:500])
work_filenames.extend(categories["dog"][:500])
np.random.shuffle(work_filenames)

In [ ]:
# таким же образом получаем и тестовую выборку
test_filenames = copy.copy(categories["cat"][500:1500])
test_filenames.extend(categories["dog"][500:1500])
np.random.shuffle(test_filenames)

In [ ]:
# получаем непосредственно тренировочную выборку
data = []
answers = []
for filename in work_filenames:
    data.append(get_image("/kaggle/working/train_extracted/train/" + filename)[1][0])
    answers.append(np.array(category_encoding[filename.split(".")[0]]))

data = np.array(data).astype('float32')
answers = np.array(answers)

In [ ]:
# то же самое и для теста
test_data = []
test_answers = []
for filename in test_filenames:
    test_data.append(get_image("/kaggle/working/train_extracted/train/" + filename)[1][0])
    test_answers.append(np.array(category_encoding[filename.split(".")[0]]))

test_data = np.array(test_data).astype('float32')
test_answers = np.array(test_answers)

In [ ]:
# достаём vgg
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

In [ ]:
# накручиваем сверху слой
class_layer = keras.layers.Dense(1, activation="sigmoid")
out = class_layer(vgg.layers[-2].output)
model = keras.Model(vgg.input, out)
model.summary()

In [ ]:
# замораживаем слои
for l, layer in enumerate(model.layers[:-1]):
    layer.trainable = False

# размораживаем последний слой
for l, layer in enumerate(model.layers[-1:]):
    layer.trainable = True

In [ ]:
# компилируем
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
%%time
# обучаем
model.fit(data, answers, epochs=3)

In [ ]:
# смотрим точность
loss, accuracy = model.evaluate(test_data, test_answers)

In [ ]:
print(accuracy)

Как видно, VGG16 позволяет достаточно быстро дообучить нейросеть для какой-то конкретной задачи и получить высокую точность.